## Feature extraction
Local Binary Patterns (LBP) và Histogram of Oriented Gradients (HOG). LBP tạo ra các đặc trưng chủ yếu liên quan đến kết cấu và mô hình bề mặt, OG tạo ra các đặc trưng liên quan đến định hướng gradient, giúp mô tả các đặc trưng hình học và cấu trúc của đối tượng trong ảnh.

In [ ]:

# Đường dẫn dữ liệu
data_dir = "DATA_split\\train"  # Thư mục chứa 4 loại rác

# Các thông số LBP và HOG
radius = 1  # Radius cho LBP
n_points = 8 * radius  # Số điểm mẫu trong LBP
orientations = 9  # Số hướng trong HOG
pixels_per_cell = (8, 8)  # Kích thước ô trong HOG
cells_per_block = (2, 2)  # Số ô trong block HOG

# Hàm tính đặc trưng LBP, HOG, Độ sáng trung bình, phương sai và histogram màu
def extract_features(image_path):
    # Đọc ảnh
    img = cv.imread(image_path)
    # Chuyển ảnh sang grayscale
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # 1. LBP
    lbp = local_binary_pattern(gray, n_points, radius, method='uniform')
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_hist = lbp_hist.astype('float')
    lbp_hist /= (lbp_hist.sum() + 1e-6)  # Chuẩn hóa
    
    # 2. HOG
    fd, hog_image = hog(gray, orientations=orientations, pixels_per_cell=pixels_per_cell,
                        cells_per_block=cells_per_block, visualize=True)
    fd = fd / np.linalg.norm(fd)  # Chuẩn hóa HOG
    
    # 3. Độ sáng trung bình và phương sai
    mean_intensity = np.mean(gray)
    std_intensity = np.std(gray)
    
    # 4. Histogram màu (RGB)
    hist_b = cv.calcHist([img], [0], None, [256], [0, 256])  # Kênh Blue
    hist_g = cv.calcHist([img], [1], None, [256], [0, 256])  # Kênh Green
    hist_r = cv.calcHist([img], [2], None, [256], [0, 256])  # Kênh Red
    hist_b /= hist_b.sum()  # Chuẩn hóa
    hist_g /= hist_g.sum()  # Chuẩn hóa
    hist_r /= hist_r.sum()  # Chuẩn hóa
    
    # Kết hợp tất cả các đặc trưng
    features = np.hstack([lbp_hist, fd, [mean_intensity, std_intensity], hist_b.flatten(), hist_g.flatten(), hist_r.flatten()])
    return features

# Lặp qua từng loại rác trong thư mục (4 thư mục con)
features_list = []
labels = []
for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):
        # Lặp qua các file ảnh trong mỗi thư mục con (class)
        for file in os.listdir(class_path):
            file_path = os.path.join(class_path, file)
            if os.path.isfile(file_path):
                features = extract_features(file_path)
                features_list.append(features)
                labels.append(class_name)  # Nhãn của từng loại rác

# Chuyển đổi thành mảng numpy
features_array = np.array(features_list)
labels_array = np.array(labels)

